In [47]:
import json
import time
import torch

from transformers import TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification

import numpy as np
import pandas as pd
from h3 import h3
from geopy.distance import geodesic as vincenty

from tqdm import tqdm

import pandas as pd

In [30]:
device = torch.device(1) 
device

device(type='cuda', index=1)

In [48]:
fine_tuned_lm_trained_out_dir = '/disk_2/ravi/paper/data/gpu_re_trained_models/fine_tune_lm_retrained_models/state_PUNJAB/state_out_dir/checkpoint-260000' 

In [49]:
fine_tuned_lm_trained_save_dir = '/disk_2/ravi/paper/data/gpu_re_trained_models/fine_tune_lm_retrained_models/state_PUNJAB/state_out_dir/checkpoint-260000' 

In [50]:
address_data_path = '/disk_2/ravi/paper/data/bert_data/state_wise/PUNJAB/text_with_grid/PUNJAB_with_grid_ids_test.csv'

test_df = pd.read_csv(address_data_path)
print(list(test_df), len(test_df))

['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id'] 184099


In [51]:
pre_trained_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_lm_trained_save_dir)

In [52]:
tuned_model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_lm_trained_out_dir).to(device)


In [53]:
tuned_model.device

device(type='cuda', index=1)

In [54]:
def load_id2label(dir_local):
    save_id2label = '/home/uddeshya.singh/Models/Fine-tuned-save/id2label.json'
    with open(save_id2label, 'r') as f:
        id2label = json.load(f)
        
    return id2label

def load_label2id(dir_local):
    save_label2id = '/home/uddeshya.singh/Models/Fine-tuned-save/label2id.json'
    with open(save_label2id, 'r') as f:
        label2id = json.load(f)
    return label2id

In [55]:
id2label = load_id2label(fine_tuned_lm_trained_save_dir)
label2id = load_label2id(fine_tuned_lm_trained_save_dir)
print(len(id2label), len(id2label))


161420 161420


In [56]:
from transformers import TextClassificationPipeline

# Assuming `device` is a torch.device object
device_index = device.index if device.type == 'cuda' else -1

pipe = TextClassificationPipeline(model=tuned_model, tokenizer=pre_trained_tokenizer, device=device_index)


In [57]:

%%time
pipe(test_df.cleaned_address.tolist()[0])


CPU times: user 34.7 ms, sys: 8.36 ms, total: 43.1 ms
Wall time: 418 ms


[{'label': 'LABEL_19668', 'score': 0.8090904951095581}]

In [58]:
cleaned_address_list_test = test_df.cleaned_address.tolist()
len(cleaned_address_list_test)

184099

In [59]:
pipe.device
fault_line_list = []
for ind, a in tqdm(enumerate(cleaned_address_list_test)):
    if a is None or type(a) != str or pd.isnull(a):
        fault_line_list.append(ind)
        
print(len(fault_line_list))

184099it [00:00, 1229303.26it/s]

0


In [60]:
new_cleaned_address_list_test = []
for ind, a in tqdm(enumerate(cleaned_address_list_test)):
    if ind in fault_line_list:
        continue
    else:
        new_cleaned_address_list_test.append(a)
        
print(len(new_cleaned_address_list_test))

184099it [00:00, 2229963.33it/s]

184099


In [61]:
def process_labels(initial_predictions_local):
    pred_grid_list_local = []
    for p in tqdm(initial_predictions_local):
        label_id = p['label']
        label_no = label_id.replace('LABEL_', '')
        grid_id = id2label[label_no]
        pred_grid_list_local.append(grid_id)

    print('len(pred_grid_list_local)', len(pred_grid_list_local))
    return pred_grid_list_local
    
print('starting now ', time.strftime('%X %x %Z'))
# initial_predictions = pipe(test_df.cleaned_address.tolist())
initial_predictions = pipe(new_cleaned_address_list_test)
print('starting now ', time.strftime('%X %x %Z'))
pred_grid_list = process_labels(initial_predictions)
print('starting now ', time.strftime('%X %x %Z'))

pred_grid_list[0:5]

starting now  14:12:21 07/12/24 IST
starting now  14:32:38 07/12/24 IST


100%|██████████| 184099/184099 [00:00<00:00, 1307917.75it/s]

len(pred_grid_list_local) 184099
starting now  14:32:38 07/12/24 IST


['8a3d14716cb7fff',
 '8a3d14449aa7fff',
 '8a3d144494a7fff',
 '8a3d14449b47fff',
 '8a3d14449b4ffff']

In [62]:

def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occured", grid_id_local, e)
        return None
    
    
def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(vincenty((lat_local,lng_local),centroid_local).meters)
    except Exception as e:
        print("exception occured ", e)
        return None
    
def get_drift_for_row(row_local):
    """get drift btw pred coor and lookup coor"""
    pred_coor = get_centroid_of_grid(row_local['top_grid'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    
    return pred_drift

# def get_drift_for_row_GRIDv2(row_local):
#     """get drift btw pred coor and lookup coor"""
#     pred_coor = get_centroid_of_grid(row_local['grid_id'])
#     pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    
#     return pred_drift


def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100:0, 200:0, 500:0, 1000:0, 2000:0, "rest":0, "None":0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue
            
        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1
    
    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]
    
    accuracy_dict_per_local = {100:0, 200:0, 500:0, 1000:0, 2000:0, "rest":0, "None":0}
    for k,v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v)/max(1,(accuracy_dict_local[2000] + accuracy_dict_local["rest"]))
    
    final_accuracy_dict_local = {'num':accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

In [63]:
test_df['top_grid'] = pred_grid_list #260000
test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df.pred_drift.tolist())
test_drift_acc_dict

{'num': {100: 142314,
  200: 158240,
  500: 168484,
  1000: 172435,
  2000: 174773,
  'rest': 9326,
  'None': 0},
 'per': {100: 0.7730297285699542,
  200: 0.8595375314368899,
  500: 0.9151815056029636,
  1000: 0.9366427845887267,
  2000: 0.9493424733431469,
  'rest': 0.05065752665685311,
  'None': 0.0}}

In [26]:
test_df['top_grid'] = pred_grid_list  ### 5000
test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df.pred_drift.tolist())
test_drift_acc_dict

{'num': {100: 113599,
  200: 144228,
  500: 162349,
  1000: 168109,
  2000: 171345,
  'rest': 12754,
  'None': 0},
 'per': {100: 0.6170538677559356,
  200: 0.7834263086708781,
  500: 0.8818570443076823,
  1000: 0.9131445580910271,
  2000: 0.9307220571540312,
  'rest': 0.06927794284596875,
  'None': 0.0}}